In [1]:
import numpy as np
import pandas as pd
from gensim.models.keyedvectors import KeyedVectors
from nltk.corpus import stopwords
from sklearn.ensemble import RandomForestClassifier
import sklearn.metrics
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.svm import SVC
from sklearn.metrics import classification_report
import nltk
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [2]:
df = pd.read_csv('train_split_Depression_AVEC2017.csv',delimiter=',',encoding='utf-8')
df.head()

,Participant_ID,PHQ8_Binary,PHQ8_Score,Gender,PHQ8_NoInterest,PHQ8_Depressed,PHQ8_Sleep,PHQ8_Tired,PHQ8_Appetite,PHQ8_Failure,PHQ8_Concentrating,PHQ8_Moving
0,303,0,0,0,0,0,0.0,0,0,0,0,0
1,304,0,6,0,0,1,1.0,2,2,0,0,0
2,305,0,7,1,0,1,1.0,2,2,1,0,0
3,310,0,4,1,1,1,0.0,0,0,1,1,0
4,312,0,2,1,0,0,1.0,1,0,0,0,0


In [3]:
df2 = pd.read_csv('test_split_Depression_AVEC2017.csv',delimiter=',',encoding='utf-8')
df2.head()

,participant_ID,Gender
0,300,1
1,301,1
2,306,0
3,308,0
4,309,1


In [4]:
df3 = pd.read_csv('dev_split_Depression_AVEC2017.csv',delimiter=',',encoding='utf-8')
df3.head()

,Participant_ID,PHQ8_Binary,PHQ8_Score,Gender,PHQ8_NoInterest,PHQ8_Depressed,PHQ8_Sleep,PHQ8_Tired,PHQ8_Appetite,PHQ8_Failure,PHQ8_Concentrating,PHQ8_Moving
0,302,0,4,1,1,1,0,1,0,1,0,0
1,307,0,4,0,0,1,0,1,0,2,0,0
2,331,0,8,1,1,1,1,1,1,1,1,1
3,335,1,12,0,1,1,3,2,3,1,1,0
4,346,1,23,0,2,3,3,3,3,3,3,3


In [5]:
df4 = pd.read_csv('full_test_split.csv',delimiter=',',encoding='utf-8')
df4.head()

,Participant_ID,PHQ_Binary,PHQ_Score,Gender
0,300,0,2,1
1,301,0,3,1
2,306,0,0,0
3,308,1,22,0
4,309,1,15,1


In [6]:
dataset1 = np.array(pd.read_csv('dev_split_Depression_AVEC2017.csv',delimiter=',',encoding='utf-8'))[:, 0:2]
dataset2 = np.array(pd.read_csv('full_test_split.csv',delimiter=',',encoding='utf-8'))[:, 0:2]
dataset3 = np.array(pd.read_csv('train_split_Depression_AVEC2017.csv',delimiter=',',encoding='utf-8'))[:, 0:2]


In [7]:
dataset2.shape

(47, 2)

In [8]:
dataset = np.concatenate((dataset1, np.concatenate((dataset2, dataset3))))


In [9]:
dataset.shape

(189, 2)

In [10]:
pos = []
neg = []

In [11]:
def checkPosNeg(dataset, index):
    for i in range(0, len(dataset)):
        if(dataset[i][0] == index):
            return dataset[i][1]
    return 0

In [12]:
for i in range(len(dataset)):
        if(dataset[i][1] == 1):
            neg.append(dataset[i][0])
        else:
            pos.append(dataset[i][0])
            
pos = np.array(pos)
neg = np.array(neg)

In [13]:
pos.shape, neg.shape

((133,), (56,))

In [14]:
df = pd.read_csv("300_TRANSCRIPT.csv",delimiter='\t',encoding='utf-8')
df.head()

,start_time,stop_time,speaker,value
0,36.588,39.668,Ellie,hi i'm ellie thanks for coming in today
1,39.888,43.378,Ellie,i was created to talk to people in a safe and ...
2,43.728,48.498,Ellie,think of me as a friend i don't judge i can't ...
3,49.188,52.388,Ellie,i'm here to learn about people and would love ...
4,52.658,58.958,Ellie,i'll ask a few questions to get us started and...


In [15]:
Data = []
Y = []
Data_test = []
Y_test = []
index = -1
for i in range(0, len(dataset3)):
    val = checkPosNeg(dataset, dataset3[i][0])
    Y.append(val)
    
    fileName = str(int(dataset3[i][0])) + "_TRANSCRIPT.csv"
    Data.append(np.array(pd.read_csv(fileName,delimiter='\t',encoding='utf-8'))[:, 2:4])


for i in range(0, len(dataset1)):
    val = checkPosNeg(dataset, dataset1[i][0])
    Y.append(val)
    fileName = str(int(dataset1[i][0])) + "_TRANSCRIPT.csv"
    Data.append(np.array(pd.read_csv(fileName,delimiter='\t',encoding='utf-8'))[:, 2:4])


In [16]:
len(Y), len(Data)

(142, 142)

In [17]:
for i in range(0, len(dataset2)):
    Y_test.append(checkPosNeg(dataset, dataset2[i][0]))
    fileName = str(int(dataset2[i][0])) + "_TRANSCRIPT.csv"
    Data_test.append(np.array(pd.read_csv(fileName,delimiter='\t',encoding='utf-8'))[:, 2:4])


In [18]:
len(Y_test), len(Data_test)

(47, 47)

In [19]:
Y = np.array(Y)
Data2 = []

Data2_test = []
Y_test = np.array(Y_test)

In [20]:
for i in range(0, len(Data)):
    script = []
    for k in range(1, len(Data[i])):
        if(Data[i][k][0] == "Participant"):
            script.append(Data[i][k][1])
    Data2.append(script)
    
for i in range(0, len(Data_test)):
    script = []
    for k in range(1, len(Data_test[i])):
        if(Data_test[i][k][0] == "Participant"):
            script.append(Data_test[i][k][1])
    Data2_test.append(script)
        
Data2 = np.array(Data2)
Data2_test = np.array(Data2_test)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:15: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  from ipykernel import kernelapp as app
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:16: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  app.launch_new_instance()


In [21]:
# def remove_context_symbol(text):
#     import re
#     return re.sub('<[^>]+>', '', text)

In [22]:
len(Data2)

142

In [23]:
# d = {'%s' % i: [i + 1] for i in range(len(Data2))}

In [24]:
# for n in range(len(Data2)):
#   for k in range(len(Data2[n])):
#     d[str(n)] = (remove_context_symbol(Data2[n]))

In [25]:
from gensim.models import Word2Vec
import gensim.downloader as api
info = api.info()
model = api.load("glove-twitter-25")
stop_words = set(stopwords.words('english'))


KeyboardInterrupt: ignored

In [ ]:
def remove_StopWOrds(sentence):
    filtered_sentence = [] 
    for w in sentence: 
        if w not in stop_words: 
            filtered_sentence.append(w)
    
    return filtered_sentence


In [ ]:
max_num_words = 15
max_num_sentence = 200

In [ ]:
len(Data2_test), len(Data2)

In [ ]:
#train_data
Array = np.zeros((Data2.shape[0], max_num_sentence, max_num_words))
print(Array.shape)
for n in range(Data2.shape[0]):
    for i in range(min(max_num_sentence, len(Data2[n][1]))):
    	sentence = Data2[n][i].split(" ")
    	sentence = remove_StopWOrds(sentence)
    	for j in range(min(max_num_words, len(sentence))):
    		try:
    			Array[n][i][j] = np.average(np.array(model[sentence[j]]))
    		except:
    			continue
X = np.zeros((len(Data2), max_num_sentence*max_num_words))
for i in range(len(Array)):
    X[i] = Array[i].flatten().reshape(1, -1)

In [ ]:
X

In [ ]:
Array_test = np.zeros((Data2_test.shape[0], max_num_sentence, max_num_words))
print(Array_test.shape)
for k in range(Data2_test.shape[0]):
    for i in range(min(max_num_sentence, len(Data2_test[k][0]))):
    	sentence = Data2_test[k][i].split(" ")
    	sentence = remove_StopWOrds(sentence)
    	for j in range(min(max_num_words, len(sentence))):
    		try:
    			Array_test[k][i][j] = np.average(np.array(model[sentence[j]]))
    		except:
    			continue
X_test = np.zeros((len(Data2_test), max_num_sentence*max_num_words))
for i in range(len(Array_test)):
    X_test[i] = Array_test[i].flatten().reshape(1, -1)

In [ ]:
X.shape, Y.shape

In [ ]:
Y_test

In [ ]:
type(X)

In [ ]:
rfModel = RandomForestClassifier()
rfModel.fit(X, Y)
#rfModel.score(X,Y)
rfModel.predict(X_test)
  



In [ ]:
rfModel.score(X,Y)


In [ ]:
# tfid3 = TfidfVectorizer(ngram_range=(1,2))
# nb3 = MultinomialNB(alpha = 1.0)
# tfid3.fit(X)
# vector3 = tfid3.transform(X)
# nb3.fit(vector3, Y)



In [ ]:
# coefs = pd.Series(nb3.coef_[0], index = tfid3.get_feature_names())
# coefs.sort_values(ascending=False).head(30)

In [ ]:
rfModel.score(X,Y)

In [ ]:
Y_pred1 

In [ ]:
np.unique(Y_pred1, return_counts = True)